In [ ]:
import pandas as pd
import unicodedata
import requests
from bs4 import BeautifulSoup

df_main = pd.read_csv('ESTANDAR_BASICA_I2324.csv')
main_mx = pd.read_csv("df_high_mx_with_turno_nivel.csv")

In [ ]:

def findIdWithName(df, name):
    cleanName = name.strip().strip('"')
    res = df[df["n_cct_clean"] == cleanName]
    if not res.empty:
        return res.iloc[0]["clavecct"]
    else:
        return None 

def findTurnWithClaveCCT(df, cct):
    res = df[df["clavecct"] == cct]
    if not res.empty:
        return res.iloc[0]["n_turno"]
    else:
        return None 

def findNivelWithClaveCCT(df, cct):
    res = df[df["clavecct"] == cct]
    if not res.empty:
        return res.iloc[0]["nivel"]
    else:
        return None 
df = pd.read_csv("ESTANDAR_BASICA_I2324.csv")
# df['n_cct_clean'] = df['n_cct'].astype(str).str.strip().str.strip('"')
# print(findIdWithName(df,"ABC KIDS ELEMENTARY SCHOOL"))

main_mx = pd.read_csv("df_high_mx_with_turno_nivel.csv")
main_mx["n_turn"] = main_mx["clavecct"].apply(lambda x: findTurnWithClaveCCT(df, x))
main_mx["nivel"] = main_mx["clavecct"].apply(lambda x: findNivelWithClaveCCT(df, x))

main_mx.to_csv("df_high_mx_with_turno_nivel.csv", index=False)



In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

# def normalize_string(txt):
#     cleanTXT = txt.strip().strip('"')
#     cleanTXT = ''.join(
#         c for c in unicodedata.normalize('NFKD', cleanTXT)
#         if not unicodedata.combining(c)
#     )
#     return cleanTXT.lower().replace(' ', '-')

def normalize_string(txt):
    cleanTXT = txt.strip().strip('"')
    cleanTXT = cleanTXT.replace("'", "").replace("’", "").replace("´", "")
    cleanTXT = ''.join(
        c for c in unicodedata.normalize('NFKD', cleanTXT)
        if not unicodedata.combining(c)
    )

    return cleanTXT.lower().replace(' ', '-')

def findDataInEscuelasMexicoOrg(schoolName,educationType,turn,state):

    url = f"https://www.escuelasmexico.org/{normalize_string(schoolName)}-{normalize_string(educationType)}-{normalize_string(turn)}-{normalize_string(state)}"
    try:
        response = requests.get(url, timeout=(5, 10))
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        li_telefone = soup.find('li', string=lambda text: text and 'Teléfono:' in text)

        if not li_telefone:
            strong_tel = soup.find('strong', string='Teléfono:')
            if strong_tel and strong_tel.parent.name == 'li':
                li_telefone = strong_tel.parent

        if li_telefone:
            texto = li_telefone.get_text(strip=True)
            numero = texto.replace('Teléfono:', '').strip()
            print(f"[v] Numero: {numero} -> ({url})")
            return numero
        else:
            print(f"[X] ERRO-> ({url})")
            return None

    except requests.exceptions.RequestException as e:
        print(f"[ERROR] mission fail {e}")
        return None
    
# findDataInEscuelasMexicoOrg("BRIGHT HOUSE","PREESCOLAR","matutino","CIUDAD DE MÉXICO")

# main_mx["telefone_escuelaMexicoOrg"] = main_mx.apply(lambda row: findDataInEscuelasMexicoOrg(
#     row["name"], row["nivel"], row["turno"], row["n_entidad"]
# ), axis=1)

mmx = pd.read_csv("df_high_mx_with_turno_nivel.csv")
print("[INFO] Start find number in escuelamexico.org")
mmx["telefone_escuelaMexicoOrg"] = mmx.apply(lambda row: findDataInEscuelasMexicoOrg(
    row["name"], row["nivel"], row["n_turn"], row["n_entidad"]
), axis=1)
mmx.to_csv("df_high_mx_with_turno_nivel_telefone_mexico_org.csv", index=False)
print("[INFO] create file")

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

def normalize_string(txt):
    cleanTXT = txt.strip().strip('"')
    cleanTXT = ''.join(
        c for c in unicodedata.normalize('NFKD', cleanTXT)
        if not unicodedata.combining(c)
    )
    return cleanTXT.lower().replace(' ', '-')

def findDataInEscuelasMexicoOrg(schoolName,educationType,turn,state):

    url = f"https://www.escuelasmexico.org/{normalize_string(schoolName)}-{normalize_string(educationType)}-{normalize_string(turn)}-{normalize_string(state)}"
    try:
        response = requests.get(url, timeout=(5, 10))
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        li_telefone = soup.find('li', string=lambda text: text and 'Teléfono:' in text)

        if not li_telefone:
            strong_tel = soup.find('strong', string='Teléfono:')
            if strong_tel and strong_tel.parent.name == 'li':
                li_telefone = strong_tel.parent

        if li_telefone:
            texto = li_telefone.get_text(strip=True)
            numero = texto.replace('Teléfono:', '').strip()
            print(f"[v] Numero: {numero} -> ({url})")
            return numero
        else:
            print(f"[X] ERRO-> ({url})")
            return None

    except requests.exceptions.RequestException as e:
        print(f"[ERROR] mission fail {e}")
        return None
    
# findDataInEscuelasMexicoOrg("BRIGHT HOUSE","PREESCOLAR","matutino","CIUDAD DE MÉXICO")

# main_mx["telefone_escuelaMexicoOrg"] = main_mx.apply(lambda row: findDataInEscuelasMexicoOrg(
#     row["name"], row["nivel"], row["turno"], row["n_entidad"]
# ), axis=1)

mmx = pd.read_csv("df_high_mx_with_turno_nivel.csv")
print("[INFO] Start find number in escuelamexico.org")
mmx["telefone_escuelaMexicoOrg"] = mmx.apply(lambda row: findDataInEscuelasMexicoOrg(
    row["name"], row["nivel"], row["n_turn"], row["n_entidad"]
), axis=1)
mmx.to_csv("df_high_mx_with_turno_nivel_telefone_mexico_org.csv", index=False)
print("[INFO] create file")